In [ ]:
pip install langchain langchain-community langchain-text-splitters langchain-huggingface langchain-groq chromadb sentence-transformers unstructured


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 1.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 11.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 46.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 90.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 72.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.9/134.9 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 65.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 78.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import os
from langchain_community.document_loaders import UnstructuredURLLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_groq import ChatGroq
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

GROQ_API_KEY = "your_api_key"
PERSIST_DIR = "./chroma_db"
URL = "https://en.wikipedia.org/wiki/San_Francisco"

# 1. Load documents
loader = UnstructuredURLLoader(urls=[URL])
docs = loader.load()

splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = splitter.split_documents(docs)

# 2. Embeddings + Chroma
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectordb = Chroma(persist_directory=PERSIST_DIR, embedding_function=embeddings)
vectordb.add_documents(chunks)
vectordb.persist()

# 3. Retriever
retriever = vectordb.as_retriever(search_kwargs={"k": 3})

# 4. Base LLM
llm = ChatGroq(
    api_key=GROQ_API_KEY,
    model="qwen/qwen3-32b",
    temperature=0,
)

# --- Self-RAG: Answer + Critique + Refine ---

# Step A: Initial Answer
answer_prompt = PromptTemplate(
    template="""
You are a helpful assistant.
Use the following context to answer the question:

Context:
{context}

Question: {question}

Answer step by step and clearly.
""",
    input_variables=["context", "question"],
)
answer_chain = LLMChain(llm=llm, prompt=answer_prompt)

# Step B: Reflection (Critique)
reflection_prompt = PromptTemplate(
    template="""
You are a critic reviewing an answer for correctness, completeness, and hallucinations.
Question: {question}
Answer: {answer}

Write a short critique.
""",
    input_variables=["question", "answer"],
)
reflection_chain = LLMChain(llm=llm, prompt=reflection_prompt)

# Step C: Refined Answer
refine_prompt = PromptTemplate(
    template="""
You are a careful assistant.
Given the question, context, an initial answer, and a critique, write an improved final answer.

Question: {question}
Context: {context}
Initial Answer: {answer}
Critique: {critique}

Final Answer:
""",
    input_variables=["question", "context", "answer", "critique"],
)
refine_chain = LLMChain(llm=llm, prompt=refine_prompt)

# ---- Run Self-RAG ----
query = "Explain the history of San Francisco."

retrieved_docs = retriever.get_relevant_documents(query)
context = "\n".join([d.page_content for d in retrieved_docs])

# Step A
initial_answer = answer_chain.run({"context": context, "question": query})

# Step B
critique = reflection_chain.run({"question": query, "answer": initial_answer})

# Step C
final_answer = refine_chain.run({
    "question": query,
    "context": context,
    "answer": initial_answer,
    "critique": critique
})

print("\n--- Iteration 1 ---\n")
print(initial_answer)

print("\n--- Iteration 2 (Critique) ---\n")
print(critique)

print("\n--- Iteration 3 (Refined Answer) ---\n")
print(final_answer)


/tmp/ipython-input-4034223889.py:54: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  answer_chain = LLMChain(llm=llm, prompt=answer_prompt)
/tmp/ipython-input-4034223889.py:89: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retrieved_docs = retriever.get_relevant_documents(query)
/tmp/ipython-input-4034223889.py:93: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  initial_answer = answer_chain.run({"context": context, "question": query})



--- Iteration 1 ---

<think>
Okay, the user is asking me to explain the history of San Francisco. Let me start by recalling what I know about San Francisco's history. First, I should mention the indigenous peoples who lived there before European contact. The Ohlone people were the original inhabitants. Then, the Spanish arrived in the 18th century, establishing missions like Mission Dolores. That's part of the Spanish colonization period.

Next, the Mexican era after Mexico gained independence from Spain. The area became part of Mexico, and there was a land grant system. Then, the Gold Rush in 1848-1849 brought a huge influx of people, leading to the city's rapid growth. San Francisco became a major port and commercial hub.

The 1906 earthquake and fire were devastating, but the city rebuilt. The 1960s counterculture movement, the Summer of Love, and the LGBTQ+ rights movement in the Castro district are important. Also, the tech boom in the late 20th and early 21st centuries with Sili